In [1]:
import requests
import json
import os
import csv
import pandas as pd
MY_API_KEY=''
path_file="queries.csv"

def write_data(new_data):
    with open('data.json','r') as json_file: 
        if(os.stat("data.json").st_size !=0):
            data_json = json.load(json_file)
            data_json.append(new_data["search-results"]["entry"])
        else:
            data_json=new_data["search-results"]["entry"]    
    with open('data.json', 'w') as outfile:
        json.dump(data_json, outfile)

def get_chunk(key,start,count='25',date='2010-2020'):
    resp = requests.get(f"https://api.elsevier.com/content/search/scopus?query={key}&start={start}&count={count}&date={date}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
    data=json.loads(resp.text.encode('utf-8'))
    return data

def get_queries(path_file):
    df=pd.read_csv(path_file)
    return df.to_dict(orient='records')

def compose_query(fields_query):
    if('management' in fields_query and 'iot' in fields_query):
        query=f"{fields_query['filter']}%28{fields_query['management']}+AND+{fields_query['iot']}%29"
    elif('management' in fields_query and 'iot' not in fields_query):
        query=f"{fields_query['filter']}%28{fields_query['management']}%29"
    elif('management' not in fields_query and 'iot' in fields_query):
        query=f"{fields_query['filter']}%28{fields_query['iot']}%29"
    return query

def write_result(path_file,fields_query,result):
    df=pd.read_csv(path_file)
    df.loc[(df['filter'] == fields_query['filter']) & (df['management']==fields_query['management']) & 
              (df['iot']==fields_query['iot']),'result']=result

    df.to_csv(path_file,index=False,)
    
def main():        
        queries=get_queries(path_file)
        for query in queries:
            composed_query=compose_query(query)
            data=get_chunk(composed_query,start=0,count=25)
            total_results=int(data['search-results']['opensearch:totalResults'])
            write_result(path_file,query,total_results)
        #data=get_chunk("TITLE%28management+AND+smart+production%29",start=0,count=25)
        #print(data)
        #total_results=int(data['search-results']['opensearch:totalResults'])
        #print(f'total results {total_results}')
        #i=25
        #write_data(data)
        #while(i<total_results):
         #   data=get_chunk("KEY%28industry+4.0%29",start=i)
         #   write_data(data)
         #   print(i)
         #   i=i+25
            

if __name__ == "__main__":
    main()



In [131]:
188584+167553

356137